In [1771]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV, KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import random
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense

In [1772]:
#불러오기
train = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')
submit = pd.read_csv('dataset/sample_submission.csv')

In [1773]:
data = pd.concat([train, test], axis = 0) #train과 test 합치기

In [1774]:
data.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0


In [1775]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36457 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          36457 non-null  int64  
 1   gender         36457 non-null  object 
 2   car            36457 non-null  object 
 3   reality        36457 non-null  object 
 4   child_num      36457 non-null  int64  
 5   income_total   36457 non-null  float64
 6   income_type    36457 non-null  object 
 7   edu_type       36457 non-null  object 
 8   family_type    36457 non-null  object 
 9   house_type     36457 non-null  object 
 10  DAYS_BIRTH     36457 non-null  int64  
 11  DAYS_EMPLOYED  36457 non-null  int64  
 12  FLAG_MOBIL     36457 non-null  int64  
 13  work_phone     36457 non-null  int64  
 14  phone          36457 non-null  int64  
 15  email          36457 non-null  int64  
 16  occyp_type     25134 non-null  object 
 17  family_size    36457 non-null  float64
 18  begin_m

In [1776]:
data['occyp_type'].describe()

count        25134
unique          18
top       Laborers
freq          6211
Name: occyp_type, dtype: object

In [1777]:
data.isnull().sum() #occyp_type 30%가 결측치

index                0
gender               0
car                  0
reality              0
child_num            0
income_total         0
income_type          0
edu_type             0
family_type          0
house_type           0
DAYS_BIRTH           0
DAYS_EMPLOYED        0
FLAG_MOBIL           0
work_phone           0
phone                0
email                0
occyp_type       11323
family_size          0
begin_month          0
credit           10000
dtype: int64

In [1778]:
data=data.drop(['index', 'FLAG_MOBIL'], axis=1)

In [1779]:
data['occyp_type'].fillna("Laborers", inplace=True) #최빈값으로 채우기

In [1780]:
data.isnull().sum()

gender               0
car                  0
reality              0
child_num            0
income_total         0
income_type          0
edu_type             0
family_type          0
house_type           0
DAYS_BIRTH           0
DAYS_EMPLOYED        0
work_phone           0
phone                0
email                0
occyp_type           0
family_size          0
begin_month          0
credit           10000
dtype: int64

# 열 삭제

In [1781]:
'''data.drop(['email'],axis=1, inplace=True)
data.drop(['phone'],axis=1, inplace=True)
data.drop(['work_phone'],axis=1, inplace=True)
data.drop(['gender'],axis=1, inplace=True)
data.drop(['reality'],axis=1, inplace=True)'''

"data.drop(['email'],axis=1, inplace=True)\ndata.drop(['phone'],axis=1, inplace=True)\ndata.drop(['work_phone'],axis=1, inplace=True)\ndata.drop(['gender'],axis=1, inplace=True)\ndata.drop(['reality'],axis=1, inplace=True)"

In [1782]:
unique_len = data.apply(lambda x : len(x.unique())) #모든 열의 요소개수가 몇개인지 확인하여 unique_len에 저장

In [1783]:
unique_len

gender              2
car                 2
reality             2
child_num           9
income_total      265
income_type         5
edu_type            5
family_type         5
house_type          6
DAYS_BIRTH       7183
DAYS_EMPLOYED    3640
work_phone          2
phone               2
email               2
occyp_type         18
family_size        10
begin_month        61
credit              4
dtype: int64

In [1784]:
group_1 = unique_len[unique_len <= 2].index #요소 개수가 2개 이하인 열의 인덱스
group_2 = unique_len[(unique_len > 2)\
             & (unique_len <= 18)].index #요소 개수가 2개 초과, 18개이하인 열의 인덱스
group_3 = unique_len[(unique_len > 18)].index #요소 개수가 18개 초과인 열의 인덱스

# group1

In [1785]:
data['gender'] = data['gender'].replace(['F','M'], [0,  1]) #gender의 f와m을 0과1로 변환, 문자는 모델학습에 사용할 수 없다.
data['car'] = data['car'].replace(['N', 'Y'], [0, 1]) #마찬가지로 숫자 0과1로 변환
data['reality'] = data['reality'].replace(['N', 'Y'], [0, 1])#마찬가지로 숫자 0과1로 변환
data[group_1] #group_1 모두 숫자로 변환완료

,gender,car,reality,work_phone,phone,email
0,0,0,0,0,0,0
1,0,0,1,0,0,1
2,1,1,1,0,1,0
3,0,0,1,0,1,0
4,0,1,1,0,0,0
...,...,...,...,...,...,...
9995,0,1,1,1,1,0
9996,1,1,1,1,0,0
9997,0,0,1,0,0,0
9998,0,1,0,0,1,0


# group2

In [1786]:
data = data.astype({'DAYS_BIRTH': 'int'})
data = data.astype({'DAYS_EMPLOYED': 'int'})
data = data.astype({'begin_month': 'int'})

In [1787]:
from sklearn import preprocessing

In [1788]:
label_encoder = preprocessing.LabelEncoder() #문자열로 이루어져 있는 값을 숫자로 인코딩해 준다

In [1789]:
data[group_2].apply(lambda x : len(x.unique())) #몇개의 데이터로 이루어져 있는지 확인

child_num       9
income_type     5
edu_type        5
family_type     5
house_type      6
occyp_type     18
family_size    10
credit          4
dtype: int64

In [1790]:
data['income_type'] = label_encoder.fit_transform(data['income_type']) #문자열로 이루어진 열 모두 숫자로 인코딩해 준다.
data['edu_type'] = label_encoder.fit_transform(data['edu_type'])
data['family_type'] = label_encoder.fit_transform(data['family_type'])
data['house_type'] = label_encoder.fit_transform(data['house_type'])
data['occyp_type'] = label_encoder.fit_transform(data['occyp_type'])

### ▲성능 안좋으면 각각 분리로 바꾸기

In [1791]:
data[group_2] #group_2 모두 숫자로 변환 완료

,child_num,income_type,edu_type,family_type,house_type,occyp_type,family_size,credit
0,0,0,1,1,2,8,2.0,1.0
1,1,0,4,0,1,8,3.0,1.0
2,0,4,1,1,1,10,2.0,2.0
3,0,0,4,1,1,14,2.0,0.0
4,0,2,1,1,1,10,2.0,2.0
...,...,...,...,...,...,...,...,...
9995,0,4,2,1,1,0,2.0,NaN
9996,0,4,4,0,1,8,2.0,NaN
9997,0,4,4,1,1,11,2.0,NaN
9998,0,0,4,1,1,8,2.0,NaN


# group3

In [1792]:
# 마이너스 값 변환
def minus(x):
    return x * -1
data['begin_month'] = data['begin_month'].apply(minus)

In [1793]:
# 나이 변환
def days_to_age(x):
    return (x*-1)/365
data['DAYS_BIRTH'] = data['DAYS_BIRTH'].apply(days_to_age)

In [1794]:
def make_bin(array, n): #숫자로 변환하는 함수를 남은 변수에 적용
    array = - array #양수 값으로 변환
    _, bin_dividers = np.histogram(array, bins = n) #count는 사용하지 않을것이므로 _로 표시
    cut_categories = pd.cut(array, bin_dividers, labels = [i for i in range(n)], include_lowest=True) #구간별로 나눈다.
    bined_array = pd.factorize(cut_categories)[0] #정수 자료형으로 변환
    return bined_array

In [1795]:
data['DAYS_BIRTH'] = make_bin(data['DAYS_BIRTH'], 10)
data['DAYS_EMPLOYED'] = make_bin(data['DAYS_EMPLOYED'], 10)
data['begin_month'] = make_bin(data['begin_month'], 10)
data['income_total'] = make_bin(data['income_total'], 10)
#모든 열에 적용

In [1796]:
data['DAYS_BIRTH']

0       0
1       1
2       2
3       3
4       3
       ..
9995    2
9996    5
9997    8
9998    4
9999    7
Name: DAYS_BIRTH, Length: 36457, dtype: int64

In [1797]:
data.dtypes

gender             int64
car                int64
reality            int64
child_num          int64
income_total       int64
income_type        int32
edu_type           int32
family_type        int32
house_type         int32
DAYS_BIRTH         int64
DAYS_EMPLOYED      int64
work_phone         int64
phone              int64
email              int64
occyp_type         int32
family_size      float64
begin_month        int64
credit           float64
dtype: object

In [1798]:
data.head()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,0,0,0,0,0,1,1,2,0,0,0,0,0,8,2.0,0,1.0
1,0,0,1,1,0,0,4,0,1,1,0,0,0,1,8,3.0,1,1.0
2,1,1,1,0,1,4,1,1,1,2,0,0,1,0,10,2.0,2,2.0
3,0,0,1,0,0,0,4,1,1,3,0,0,1,0,14,2.0,3,0.0
4,0,1,1,0,2,2,1,1,1,3,0,0,0,0,10,2.0,4,2.0


In [1799]:
object_col=['gender', 'reality','child_num','income_total',"DAYS_BIRTH",'DAYS_EMPLOYED','work_phone','phone','email','family_size','begin_month','car', 'income_type','edu_type','family_type','house_type','occyp_type']

In [1800]:
enc = OneHotEncoder()
enc.fit(data.loc[:,object_col])


data_onehot_df = pd.DataFrame(enc.transform(data.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
data.drop(object_col, axis=1, inplace=True)
data = pd.concat([data.reset_index(drop=True), data_onehot_df], axis=1)

In [1801]:
data.dtypes

credit           float64
gender_0         float64
gender_1         float64
reality_0        float64
reality_1        float64
                  ...   
occyp_type_13    float64
occyp_type_14    float64
occyp_type_15    float64
occyp_type_16    float64
occyp_type_17    float64
Length: 103, dtype: object

In [1802]:
'''data.drop(['credit_nan'], axis=1, inplace=True)'''

"data.drop(['credit_nan'], axis=1, inplace=True)"

In [1803]:
data.head()

,credit,gender_0,gender_1,reality_0,reality_1,child_num_0,child_num_1,child_num_2,child_num_3,child_num_4,...,occyp_type_8,occyp_type_9,occyp_type_10,occyp_type_11,occyp_type_12,occyp_type_13,occyp_type_14,occyp_type_15,occyp_type_16,occyp_type_17
0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,2.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# test, train 분리

In [1804]:
test = data[-10000:] #다시 test와 train 분리하기

In [1805]:
train = data[:-10000]

In [1806]:
'''test.drop(['credit_0.0','credit_1.0','credit_2.0' ], axis=1, inplace=True)'''
test.drop(['credit'], axis=1, inplace=True)

In [1807]:
np_train=train.to_numpy()
np_train.shape

(26457, 103)

In [1808]:
np_test = test.to_numpy()
np_test.shape

(10000, 102)

In [1809]:
np_result = np_train[:,-1]
np_result.shape

(26457,)

In [1810]:
np_result

array([0., 0., 0., ..., 0., 0., 0.])

In [1811]:
np_train = np.delete(np_train, -1, axis = 1)

In [1812]:
np_train.shape

(26457, 102)

In [1813]:
np_train, np_val= train_test_split(np_train, test_size=0.3, shuffle=False, random_state=1004)
#train을 train과 valid로 분리

In [1814]:
np_train_result, np_val_result= train_test_split(np_result, test_size=0.3, shuffle=False, random_state=1004)
#결과를 train결과와 valid결과로 분리

In [1815]:
np_result.shape

(26457,)

In [1816]:
'''np_train = np.delete(np_train , (102,103,104), axis = 1)
np_val = np.delete(np_val , (102,103,104), axis = 1)'''


'np_train = np.delete(np_train , (102,103,104), axis = 1)\nnp_val = np.delete(np_val , (102,103,104), axis = 1)'

In [1817]:
np_train.shape

(18519, 102)

In [1818]:
np_train.size

1888938

In [1819]:
np_val.shape

(7938, 102)

In [1820]:
np_val_result.shape

(7938,)

In [1821]:
class creditModel(keras.Model):
    def __init__(self):
        super(creditModel, self).__init__()
        self.dense1 = keras.layers.Dense(210,input_dim = 2, activation = 'relu')
        self.dense2 = keras.layers.Dense(102,  activation = 'relu')
        self.dense3 = keras.layers.Dense(51,  activation = 'relu')
        self.dense4 = keras.layers.Dense(3, activation = 'softmax')

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense4(x)

In [1822]:
np_train_result = keras.utils.to_categorical(np_train_result, num_classes = 3)

In [1823]:
model = creditModel()
model.compile(optimizer = keras.optimizers.Adam(learning_rate = 0.5), loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

In [ ]:
model.fit(np_train, np_train_result, epochs = 3000, batch_size=10)
model.evaluate(np_val, np_val_result, batch_size=4,steps=1000)

Epoch 1/3000
1852/1852 [==============================] - 2s 839us/step - loss: 1.5734 - categorical_accuracy: 0.9888
Epoch 2/3000
1852/1852 [==============================] - 2s 828us/step - loss: 0.1669 - categorical_accuracy: 0.9941
Epoch 3/3000
1852/1852 [==============================] - 2s 861us/step - loss: 0.1249 - categorical_accuracy: 0.9955
Epoch 4/3000
1852/1852 [==============================] - 2s 832us/step - loss: 0.2661 - categorical_accuracy: 0.9943
Epoch 5/3000
1852/1852 [==============================] - 2s 845us/step - loss: 0.0369 - categorical_accuracy: 0.99510s - loss: 0.0407 
Epoch 6/3000
1852/1852 [==============================] - 2s 912us/step - loss: 0.0342 - categorical_accuracy: 0.9954
Epoch 7/3000
1852/1852 [==============================] - 2s 919us/step - loss: 0.0316 - categorical_accuracy: 0.99581s -
Epoch 8/3000
1852/1852 [==============================] - 2s 901us/step - loss: 0.0359 - categorical_accuracy: 0.99530s - loss: 0.0358 - categorical_acc

1852/1852 [==============================] - 2s 881us/step - loss: 0.0287 - categorical_accuracy: 0.9957TA: 1s - loss: 0 - ETA: 0s - loss: 0.0286 - cate
Epoch 56/3000
1852/1852 [==============================] - 2s 937us/step - loss: 0.0293 - categorical_accuracy: 0.99520s - loss: 0.0293 - categorical_accuracy: 
Epoch 57/3000
1852/1852 [==============================] - 2s 858us/step - loss: 0.0335 - categorical_accuracy: 0.99580s - loss: 0.0276 - categorical_accuracy - ETA: 0s - loss: 0.0311 - categorical_accuracy: 0. - ETA: 0s - loss: 0.0324 - categorical_accuracy: 
Epoch 58/3000
1852/1852 [==============================] - 2s 852us/step - loss: 0.5953 - categorical_accuracy: 0.99311s - loss: 1.3562 - categorica - ETA: 0s - loss: 0.8896 - categorica - ETA: 0s - loss: 0.6641 - categorical_accu
Epoch 59/3000
1852/1852 [==============================] - 2s 869us/step - loss: 0.0330 - categorical_accuracy: 0.9949
Epoch 60/3000
1852/1852 [==============================] - 2s 828us/step - 

In [ ]:
submit.to_csv('submit/Guk_202105011_test_submit_ensemble.csv', index=False) # 0.7272812144

In [ ]:
submit.head(20)